In [1]:
import os
import numpy as np
from ast import literal_eval as make_tuple


directory = "."

ds_names = {"Reynolds","UK"}
all_models = []
dat = {} 
for file in os.listdir(directory):
    if file.endswith(".txt"): 
        with open(os.path.join(directory, file),"r") as f:
            for line in f:
                if line.find("y_") > -1:
                    if line.find("(") > -1 and line.find(")") > -1:
                        tpl = make_tuple(line)    
                        line = " ".join([str(t) for t in tpl])    
                    
                    for dname in ds_names:
                        start = line.find(dname)
                        if start > -1:
                            model_name = line[:start].strip()
                            all_models.append(model_name)
                            res = line[start + len(dname):].strip().split()

                            deep = res[0]
                            r2 = res[2]
                            r1 = res[1]
                            dat.setdefault(dname,{}).setdefault(deep,{}).setdefault(model_name,[r1,r2])

all_models = set(all_models)
new_dat = []

rtable = {"y_1":"5","y_2":"10","y_3":"20","y_4":"30","y_5":"40","y_6":"50","y_7":"60","y_8":"90"}
utable = {"y_1":"10","y_2":"10","y_3":"5","y_4":"5","y_5":"15","y_6":"15","y_7":"25","y_8":"25","y_9":"50","y_10":"50"}

for d in dat:
    for dep in dat[d]:
        if d == "Reynolds":
            dp =  rtable[dep]   
        else:
            if d == "UK":
                dp = utable[dep]
                
        tmp = [d, dp]
        for model in all_models:
            if model in dat[d][dep]:
                tmp += [np.sqrt(float(dat[d][dep][model][0])), float(dat[d][dep][model][1])]
            else:
                tmp += [None,None]
        new_dat.append(tmp)    


                        

In [2]:
import pandas as pd
from functools import partial

def bold_formatter(x, values, num_decimals=2):
    match = False
    for v in values:
         if round(x, num_decimals) == round(v, num_decimals):
             match = True
             break
             
    if match:
        return f"\\textbf{{{x:.{num_decimals}f}}}"
    else:
        return f"{x:.{num_decimals}f}"

pd.options.display.float_format = '{:.2f}'.format

names_top = [model for model in all_models for _ in (0, 1)]
names_bottom = [i for _ in all_models for i in ("RMSE", "MAE")]

df = pd.DataFrame(new_dat, columns=[["",""] + names_top, ["Dataset","Depth, cm"] + names_bottom])

for i,c in enumerate(df.columns):
    if i > 1:
        df[c] = pd.to_numeric(df[c])

df = df.set_index([("","Dataset"),("","Depth, cm")]).sort_index()

col2_bold = {}

for i,r in df.iterrows():
    ma_odd = r[0::2].min() 
    ma_ev =  r[1::2].min()
    ma_odd_id = 2 * r[0::2].argmin()
    ma_ev_id = 2 * r[1::2].argmin() + 1
    ma_odd = r[ma_odd_id]
    ma_ev = r[ma_ev_id]
    if not df.columns[ma_odd_id] in col2_bold:
        col2_bold[df.columns[ma_odd_id]] = []
    col2_bold[df.columns[ma_odd_id]].append(ma_odd)
    if not df.columns[ma_ev_id] in col2_bold:
        col2_bold[df.columns[ma_ev_id]] = []
    col2_bold[df.columns[ma_ev_id]].append(ma_ev)
    
fmts_max_2f = {column: partial(bold_formatter, values=col2_bold[column], num_decimals=2) for column in col2_bold}



/tmp/ipykernel_2088679/99201611.py:36: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ma_odd = r[ma_odd_id]
/tmp/ipykernel_2088679/99201611.py:37: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ma_ev = r[ma_ev_id]


In [3]:
print(df.to_latex(float_format="%.2f",formatters=fmts_max_2f))

\begin{tabular}{llrrrrrrrrrrrrrrrrrr}
\toprule
 &  & \multicolumn{2}{r}{Cascade Forest} & \multicolumn{2}{r}{GRU} & \multicolumn{2}{r}{Boosted Forest} & \multicolumn{2}{r}{ARIMA} & \multicolumn{2}{r}{ecdfr} & \multicolumn{2}{r}{AWDF} & \multicolumn{2}{r}{XGB} & \multicolumn{2}{r}{LSTM} & \multicolumn{2}{r}{TKAN} \\
 &  & RMSE & MAE & RMSE & MAE & RMSE & MAE & RMSE & MAE & RMSE & MAE & RMSE & MAE & RMSE & MAE & RMSE & MAE & RMSE & MAE \\
('', 'Dataset') & ('', 'Depth, cm') &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  \\
\midrule
\multirow[t]{8}{*}{Reynolds} & 10 & 1.81 & 1.32 & 1.89 & 1.52 & \textbf{1.49} & \textbf{1.02} & 1.50 & 1.03 & 2.33 & 1.79 & 2.93 & 2.21 & 1.58 & 1.09 & 2.08 & 1.59 & 2.66 & 1.65 \\
 & 20 & 1.23 & 0.94 & 1.95 & 1.40 & \textbf{0.99} & \textbf{0.66} & 1.15 & 0.76 & 1.69 & 1.28 & NaN & NaN & 0.99 & 0.69 & 1.78 & 1.39 & 1.25 & 0.91 \\
 & 30 & 0.89 & 0.67 & 2.41 & 1.81 & \textbf{0.55} & \textbf{0.33} & 0.74 & 0.41 & 1.32 & 0.99 & 1.17 & 0.89 & 0.60 & 0.41 & 0

In [4]:
df

Cascade Forest       GRU      Boosted Forest       \
                                    RMSE  MAE RMSE  MAE           RMSE  MAE   
(, Dataset) (, Depth, cm)                                                     
Reynolds    10                      1.81 1.32 1.89 1.52           1.49 1.02   
            20                      1.23 0.94 1.95 1.40           0.99 0.66   
            30                      0.89 0.67 2.41 1.81           0.55 0.33   
            40                      0.92 0.73 0.99 0.76           0.49 0.35   
            5                       2.33 1.46 4.26 3.19           2.32 1.34   
            50                      0.72 0.57 0.76 0.59           0.31 0.22   
            60                      0.68 0.55 1.55 1.17           0.27 0.17   
            90                      0.58 0.41 1.92 1.50           0.15 0.10   
UK          10                      0.91 0.63 0.97 0.63           0.56 0.41   
            10                      0.97 0.64 1.02 0.85           0.52 0.38   
            15                      0.84 0.57 0.79 0.58           0.45 0.34   
            15                      0.70 0.52 0.73 0.54           0.47 0.35   
            25                      0.63 0.43 0.74 0.49           0.31 0.22   
            25                      0.60 0.44 0.68 0.49           0.32 0.23   
            5                       0.92 0.68 1.15 0.89           0.75 0.55   
            5                       1.24 0.75 1.43 0.85           0.80 0.57   
            50                      0.68 0.47 1.03 0.65           0.24 0.15   
            50                      0.62 0.44 0.77 0.55           0.29 0.19   

                          ARIMA      ecdfr      AWDF       XGB      LSTM       \
                           RMSE  MAE  RMSE  MAE RMSE  MAE RMSE  MAE RMSE  MAE   
(, Dataset) (, Depth, cm)                                                       
Reynolds    10             1.50 1.03  2.33 1.79 2.93 2.21 1.58 1.09 2.08 1.59   
            20             1.15 0.76  1.69 1.28  NaN  NaN 0.99 0.69 1.78 1.39   
            30             0.74 0.41  1.32 0.99 1.17 0.89 0.60 0.41 0.99 0.80   
            40             0.71 0.48  1.35 0.99  NaN  NaN 0.58 0.42 0.92 0.74   
            5              2.01 1.23  2.85 1.93 3.81 2.79 2.19 1.29 3.42 2.72   
            50             0.47 0.30  1.60 1.27 1.27 1.03 0.37 0.28 1.57 1.20   
            60             0.32 0.22  1.65 1.37  NaN  NaN 0.30 0.23 1.18 0.89   
            90             0.12 0.08  1.09 0.91  NaN  NaN 0.23 0.17 1.38 1.05   
UK          10             0.69 0.49   NaN  NaN  NaN  NaN 0.59 0.41 0.84 0.62   
            10             0.72 0.47   NaN  NaN  NaN  NaN 0.57 0.42 0.78 0.49   
            15             0.55 0.41   NaN  NaN  NaN  NaN 0.51 0.38 0.75 0.53   
            15             0.50 0.37   NaN  NaN  NaN  NaN 0.50 0.37 0.68 0.51   
            25             0.38 0.27   NaN  NaN  NaN  NaN 0.34 0.26 0.99 0.72   
            25             0.36 0.26   NaN  NaN  NaN  NaN 0.35 0.26 0.75 0.59   
            5              0.86 0.64   NaN  NaN  NaN  NaN 0.77 0.57 0.87 0.66   
            5              0.85 0.63   NaN  NaN  NaN  NaN 0.84 0.58 1.15 0.75   
            50             0.32 0.18   NaN  NaN  NaN  NaN 0.26 0.17 0.57 0.47   
            50             0.37 0.19   NaN  NaN  NaN  NaN 0.28 0.18 0.79 0.58   

                          TKAN       
                          RMSE  MAE  
(, Dataset) (, Depth, cm)            
Reynolds    10            2.66 1.65  
            20            1.25 0.91  
            30            2.90 2.04  
            40            0.63 0.50  
            5             2.16 1.44  
            50            0.41 0.30  
            60            0.27 0.19  
            90            0.27 0.17  
UK          10            0.70 0.46  
            10            1.89 1.30  
            15            1.45 0.93  
            15            0.56 0.42  
            25            4.49 3.79  
            25            0.82 0.54  
            5     